### Case - Cancelamento de Clientes

Recentemente a empresa percebeu que da sua base total de clientes, a maioria são clientes inativos, ou seja, que já cancelaram o serviço.

Precisando melhorar seus resultados ela quer conseguir entender os principais motivos desses cancelamentos e quais as ações mais eficientes para reduzir esse número.

In [3]:
import pandas as pd

# importar o arquivo csv para um dataframe
tabela = pd.read_csv('cancelamentos.csv')
# Vizualizar os dados
display(tabela.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881666 entries, 0 to 881665
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   CustomerID              881664 non-null  float64
 1   idade                   881664 non-null  float64
 2   sexo                    881664 non-null  object 
 3   tempo_como_cliente      881663 non-null  float64
 4   frequencia_uso          881663 non-null  float64
 5   ligacoes_callcenter     881664 non-null  float64
 6   dias_atraso             881664 non-null  float64
 7   assinatura              881661 non-null  object 
 8   duracao_contrato        881663 non-null  object 
 9   total_gasto             881664 non-null  float64
 10  meses_ultima_interacao  881664 non-null  float64
 11  cancelou                881664 non-null  float64
dtypes: float64(9), object(3)
memory usage: 80.7+ MB


None

In [4]:
# Tratamento de Erros
    # 1. Valores vazios
    # 2. Valores informações inuteis
    


tabela = tabela.drop(['CustomerID','sexo'], axis=1)
tabela = tabela.dropna()
display(tabela.info())

<class 'pandas.core.frame.DataFrame'>
Index: 881659 entries, 0 to 881665
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   idade                   881659 non-null  float64
 1   tempo_como_cliente      881659 non-null  float64
 2   frequencia_uso          881659 non-null  float64
 3   ligacoes_callcenter     881659 non-null  float64
 4   dias_atraso             881659 non-null  float64
 5   assinatura              881659 non-null  object 
 6   duracao_contrato        881659 non-null  object 
 7   total_gasto             881659 non-null  float64
 8   meses_ultima_interacao  881659 non-null  float64
 9   cancelou                881659 non-null  float64
dtypes: float64(8), object(2)
memory usage: 74.0+ MB


None

In [5]:
# Analise de Dados entender estão os cancelamentos
display(tabela['cancelou'].value_counts())
display(tabela['cancelou'].value_counts(normalize=True).map('{:.1%}'.format))


cancelou
1.0    499993
0.0    381666
Name: count, dtype: int64

cancelou
1.0    56.7%
0.0    43.3%
Name: proportion, dtype: object

In [6]:
#Ver a media de cancelamentos por duracao de contrato

# Filtrando apenas as colunas numéricas
colunas_numericas = tabela.select_dtypes(include='float64').columns

# Calculando a média das colunas numéricas agrupadas por 'duracao_contrato'
media_por_duracao = tabela.groupby('duracao_contrato')[colunas_numericas].mean()
media_por_assinatura = tabela.groupby('assinatura')[colunas_numericas].mean()

# Exibindo o resultado
display(media_por_duracao, media_por_assinatura)



,idade,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,total_gasto,meses_ultima_interacao,cancelou
duracao_contrato,,,,,,,,
Annual,38.842165,31.446186,15.880213,3.263401,12.465156,651.697738,14.236107,0.460760
Monthly,41.552407,30.538555,15.499274,4.985649,15.007267,550.616435,15.478012,1.000000
Quarterly,38.830938,31.419916,15.886662,3.265245,12.460863,651.427783,14.234544,0.460255


,idade,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,total_gasto,meses_ultima_interacao,cancelou
assinatura,,,,,,,,
Basic,39.458756,31.935347,15.808678,3.652196,13.026310,628.673496,14.488492,0.581782
Premium,39.323023,30.883415,15.809341,3.577677,12.916779,632.927300,14.475566,0.559413
Standard,39.341109,30.976879,15.804613,3.585303,12.956423,633.134360,14.478837,0.560697


In [7]:
#A 100% de cancelamento na assunatura mensal logo vamos remover ela pois não compensa
tabela = tabela[tabela['duracao_contrato'] != 'Monthly']
display(tabela['duracao_contrato'].value_counts())
display(tabela['duracao_contrato'].value_counts(normalize = True).map('{:.1%}'.format))

duracao_contrato
Annual       354395
Quarterly    353059
Name: count, dtype: int64

duracao_contrato
Annual       50.1%
Quarterly    49.9%
Name: proportion, dtype: object

In [ ]:
import plotly.express as px

for coluna in tabela.columns:
    
    grafico = px.histogram(tabela, x=coluna, color='cancelou', text_auto=True)
    grafico.show()


In [19]:
# Clientes com > 5 ligacoes no callcenter, cancelam

tabela = tabela[tabela['ligacoes_callcenter'] < 5 ]

# Clientes com > 21 dias de atraso, cancelam
tabela = tabela[tabela['dias_atraso'] <= 20]
# clientes mensais cancelam 100% das vezes

display(tabela['cancelou'].value_counts())
display(tabela['cancelou'].value_counts(normalize = True).map('{:.1%}'.format))

cancelou
0.0    379032
1.0     85447
Name: count, dtype: int64

cancelou
0.0    81.6%
1.0    18.4%
Name: proportion, dtype: object